In [12]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
#from sklearn.linear_model import LinearRegression 
from sklearn import metrics 
#import matplotlib.pyplot as plt 
import numpy as np 
#import seaborn as sns
import mlflow
from mlflow import sklearn
from sklearn import preprocessing
from sklearn import tree

In [13]:
data = pd.read_csv('winequality-red.csv', sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
# there are no categorical variables. each feature is a number. Regression problem. 
# Given the set of values for features, we have to predict the quality of wine. finding correlation of each feature with our target variable - quality
#correlations = df.corr()['quality'].drop('quality')
#print(correlations)

y = data.quality
X = data.drop('quality', axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
191,6.4,0.37,0.25,1.9,0.074,21.0,49.0,0.99740,3.57,0.62,9.8
712,8.7,0.69,0.00,3.2,0.084,13.0,33.0,0.99920,3.36,0.45,9.4
743,11.6,0.41,0.58,2.8,0.096,25.0,101.0,1.00024,3.13,0.53,10.0
1447,6.8,0.67,0.00,1.9,0.080,22.0,39.0,0.99701,3.40,0.74,9.7
1080,10.3,0.27,0.56,1.4,0.047,3.0,8.0,0.99471,3.16,0.51,11.8


In [18]:
X_train_scaled = preprocessing.scale(X_train)
X_train_scaled

array([[-1.10285949, -0.88786188, -0.10265992, ...,  1.67076394,
        -0.22599782, -0.5805725 ],
       [ 0.21632961,  0.89003386, -1.39246378, ...,  0.29944477,
        -1.23521341, -0.95533863],
       [ 1.879655  , -0.66562491,  1.59988118, ..., -1.20247624,
        -0.76028843, -0.39318944],
       ...,
       [-0.81607925, -0.05447325, -0.10265992, ...,  0.9524539 ,
        -0.9383853 ,  0.54372587],
       [-0.58665506, -1.24899695,  0.61963025, ..., -0.15766163,
        -0.81965405,  1.01218353],
       [-0.29987482,  0.08442485, -0.77335793, ...,  0.16884294,
        -0.28536345,  1.10587506]])

In [19]:
clf=tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [20]:
confidence = clf.score(X_test, y_test)
print("\nThe confidence score:\n")
print(confidence)


The confidence score:

0.590625


In [21]:
y_pred = clf.predict(X_test)

In [23]:

#converting the numpy array to list
x=np.array(y_pred).tolist()

#printing first 5 predictions
print("\nThe prediction:\n")
for i in range(0,5):
    print( x[i])
    
#printing first five expectations
print("\nThe expectation:\n")
print( y_test.head())


The prediction:

5
7
5
7
5

The expectation:

1438    5
1002    7
221     5
1544    7
1446    5
Name: quality, dtype: int64


In [27]:
mlflow.set_experiment('WineQuality')
with mlflow.start_run():
    
    mlflow.log_metric("rmse", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    mlflow.log_metric("r2", metrics.mean_squared_error(y_test, y_pred))
    mlflow.log_metric("mae", metrics.mean_absolute_error(y_test, y_pred))

    mlflow.sklearn.log_model(clf, 'model')